# Particle Fluid

Modeling fluid motion by simulating a set of particles. 

Credit to Professor Bo Zhu, on whose Dartmouth COSC89 course notes the following code is based

### Structure: 
Give each particle an index\
each particle has a position (2D vector), velocity (2D vector), mass (scalar), density (scalar), pressure (scalar)\
Have a global constant h (kernel radius)

Have kernels to calculate w_spiky, gradient_w_spiky, laplacian_w_spiky\
Where w_spiky takes two indices (which it uses to calculate the radius r)\
grad and laplacian take just one index (which it uses to find the velocity of that index)\

Have kernels to calculate w_vis, gradient_w_vis, laplacian_w_vis\
Same inputs as for the corresponding spiky kernels

Spatial Hashing: \
Keep a dictionary hashing a vec2 -> list of indices\
Iterate over all points and add them to the array that corresponds 

Have each particle keep track of its neighbors with an array of arrays

Use explicit time integration to advance velocity and position for each time step

Have methods to update:\
Neighbors\
Density\
Pressure\
Pressure Force\
Viscosity Force\
Body Force\
Boundary Collision Force (penalty based collision respoonse)\
Temperature and Buoyancy (?)

In [2]:
import taichi as ti

ti.init()

### Global variables ###
dt = 2e-3
num_particles = 225
substeps = 10
screen_size = 800
ks = 2e2
kd = 0.5e1
radius = 5

pos = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
vel = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
force = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
r = ti.field(dtype=ti.f32, shape=num_particles)
m = ti.field(dtype=ti.f32, shape=num_particles)

### Spiky Kernel Functions ###
@ti.kernel
def w_spiky(idx1:ti.i32, idx2: ti.i32):
    pass

@ti.kernel
def gradient_w_spiky(idx1:ti.i32, idx2: ti.i32): # do I need this
    pass 

@ti.kernel
def laplacian_w_spiky(idx1:ti.i32, idx2: ti.i32):
    pass


### Viscosity Kernel Functions ###
@ti.kernel
def w_vis(idx1:ti.i32, idx2: ti.i32): # do I need this
    pass

@ti.kernel
def gradient_w_vis(idx1:ti.i32, idx2: ti.i32): # do I need this
    pass

@ti.kernel
def laplacian_w_vis(idx1:ti.i32, idx2: ti.i32):
    pass


### Spatial Hashing Functions ###
@ ti.func
def add_point(idx:ti.i32):
    pass

@ti.kernel
def update_voxels():
    # clear voxels
    # for each point, add the point
    pass


### Simulator Functions ###
@ti.func
def clear_force():
    pass

@ti.func
def update_neighbors():
    pass

@ti.func
def update_density():
    pass

@ti.func
def update_pressure():
    pass

@ti.func
def update_pressure_force():
    pass

@ti.func
def update_viscosity_force():
    pass

@ti.func
def update_body_force():
    pass

@ti.func
def update_boundary_collision_force():
    pass

@ti.kernel
def substep():
    clear_force()
    update_neighbors()
    update_density()
    update_pressure()
    update_pressure_force()
    update_viscosity_force()
    update_body_force()
    update_boundary_collision_force()
    # advance pos and vel

### Setup ###
@ti.kernel
def init_particles():
    for idx in range(num_particles):
        r[idx] = radius
        
    # initialize positions and initial velocities/ forces
    


### Driver ###
def main():
    gui = ti.GUI('Particle Fluid', (screen_size, screen_size))
        
    init_particles()
        
    while gui.running:
        pass
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


NameError: name 'new_particle' is not defined